In [133]:
#!/usr/bin/env python3

import numpy as np
import math
from collections import deque

class Node:
	"""docstring for Node"""
	def __init__(self, arr):
		self.arr = arr
		self.g = None
		self.h = None
		self.cost = None
		self.parent = None
		self.step = None

def region(path, rp_start):
    r = []
    r.append(rp_start)
    for i in range(len(path)):
        if path[i] == 'UP':
            p = [(r[i][0][0], r[i][0][1], r[i][0][2]+1), (r[i][1][0], r[i][1][1], r[i][1][2]+1), (r[i][2][0], r[i][2][1], r[i][2][2]+1), (r[i][3][0], r[i][3][1], r[i][3][2]+1)]
            r.append(p)
        if path[i] == 'DOWN':
            p = [(r[i][0][0], r[i][0][1], r[i][0][2]-1), (r[i][1][0], r[i][1][1], r[i][1][2]-1), (r[i][2][0], r[i][2][1], r[i][2][2]-1), (r[i][3][0], r[i][3][1], r[i][3][2]-1)]
            r.append(p)
        if path[i] == 'CW' or path[i] == 'CCW':
            p = [rot_point((r[i][0][0], r[i][0][1], r[i][0][2]), path[i]), rot_point((r[i][1][0], r[i][1][1], r[i][1][2]), path[i]), rot_point((r[i][2][0], r[i][2][1], r[i][2][2]), path[i]), rot_point((r[i][3][0], r[i][3][1], r[i][3][2]), path[i])]
            r.append(p)
    return r
            
def centroid(rp_start):
    return (((rp_start[0][0]+rp_start[1][0]+rp_start[2][0]+rp_start[3][0])/4, (rp_start[0][1]+rp_start[1][1]+rp_start[2][1]+rp_start[3][1])/4, (rp_start[0][2]+rp_start[1][2]+rp_start[2][2]+rp_start[3][2])/4))

def get_setup():  
	# g_p1 = [-1.25, 0, 1.5]
	# g_p2 = [0, -1.25, 1.5]
	# g_p3 = [1.25, 0, 1.5]
    
#     s_p1 = [-1.25, 0, 3.5]
#     s_p2 = [0, -1.25, 3.5]
#     s_p3 = [1.25, 0, 3.5]
    
#     g_p1 = [0, -1.25, 0.5]
#     g_p2 = [1.25, 0, 0.5]
#     g_p3 = [0, 1.25, 0.5]

    rp_start1 = [[-1.25, -1.25, -3], [-1.25, -1.25, -2], [-1.25, 1.25, -2], [-1.25, 1.25, -3]]
    rp_goal1 = [[1.25, -1.25, 2], [1.25, -1.25, 1], [1.25, 1.25, 1], [1.25, 1.25, 2]]
    
    rp_start2 = [[1.25, -1.25, -3.0], [1.25, -1.25, -2], [-1.25, -1.25, -2], [-1.25, -1.25, -3]]
    rp_goal2= [[1.25, 1.25, 2], [1.25, 1.25, 1], [-1.25, 1.25, 1], [-1.25, 1.25, 2]]
    
    rp_start3 = [[1.25, 1.25, -3], [1.25, 1.25, -2], [1.25, -1.25, -2], [1.25, -1.25, -3]]
    rp_goal3 = [[-1.25, 1.25, 2], [-1.25, 1.25, 1], [-1.25, -1.25, 1], [-1.25, -1.25, 2]]
    
    s_p1 = centroid(rp_start1)
    s_p2 = centroid(rp_start2)
    s_p3 = centroid(rp_start3)
    
    g_p1 = centroid(rp_goal1)
    g_p2 = centroid(rp_goal2)
    g_p3 = centroid(rp_goal3)
    
    start = np.array([s_p1, s_p2, s_p3])
    goal = np.array([g_p1, g_p2, g_p3])
    
    return start, goal, rp_start1, rp_start2, rp_start3, rp_goal1, rp_goal2, rp_goal3


def neighbours(s_node):
	fingers = s_node.arr
	directions = ["UP", "DOWN", "CW", "CCW"]
	n_list = []
	for d in directions:
		temp = np.array(fingers, copy=True)  
		if  d == "UP":
			if fingers[0][2] + 1 < 4:
				temp[0][2] += 1
				temp[1][2] += 1
				temp[2][2] += 1
				n_list.append((d, temp))
		if d == "DOWN":
			if fingers[0][2] - 1 > -4:
				temp[0][2] -= 1
				temp[1][2] -= 1
				temp[2][2] -= 1
				n_list.append((d, temp))
		if d == "CW" or d == "CCW":
			temp[0] = rot_point(temp[0], d)
			temp[1] = rot_point(temp[1], d)
			temp[2] = rot_point(temp[2], d)
			n_list.append((d, temp))

	return n_list

def rot_point(point, direction):
	if direction == "CW":
		theta = np.pi/2
	if direction == "CCW":
		theta = -np.pi/2
	rot_mat = np.array([[round(math.cos(theta), 2), round(-math.sin(theta), 2), 0],
                        [round(math.sin(theta), 2), round(math.cos(theta), 2), 0],
                        [0, 0, 1]])
	new_point = rot_mat.dot(point)
	return new_point

def gcost(direction):
	if direction == "UP" or direction == "DOWN":
		return 1.0
	if direction == "CW" or direction == "CCW":
		return 5.5


def astar(start, goal):
	found = False
	visit = deque([])
	q = []
	steps = 0
	path = []
	if np.array_equal(start, goal, equal_nan=False):
		return path, steps

	s_node = Node(start)
	s_node.cost = 0
	s_node.g = 0
	s_node.h = 0

	visit.append((tuple(start[0]),tuple(start[1]),tuple(start[2])))


	while not found:
		if np.array_equal(s_node.arr, goal, equal_nan=False):
			found = True
			break
		else:
			for node in neighbours(s_node):
				fingers = node[1]

				if (tuple(fingers[0]),tuple(fingers[1]),tuple(fingers[2])) not in visit:
					h_total = 0
					for i in range(len(fingers)):
						h = 0 
						h = np.sqrt((fingers[i][0]-goal[i][0])**2 + (fingers[i][1]-goal[i][1])**2 + (fingers[i][2]-goal[i][2])**2)
						h_total += h

					temp_obj = Node(fingers)
					temp_obj.h = h_total
					temp_obj.parent = s_node
					temp_obj.g = s_node.g + gcost(node[0])
					temp_obj.cost = temp_obj.g + temp_obj.h
					temp_obj.step = node[0]
					q.append((temp_obj, temp_obj.cost))
					

			q.sort(key = lambda x:x[1])
			priority = q.pop(0)
			s_node = priority[0]
			visit.append((tuple(s_node.arr[0]),tuple(s_node.arr[1]),tuple(s_node.arr[2])))

	steps = len(visit)
	while s_node.parent is not None:
		path.append(s_node.step)
		s_node = s_node.parent
	path.reverse()

	return path, (steps-1)

def points(start, goal, path):
    n_list = []

    for i in range(len(path)):
        for j in range(len(start)):
            if path[i] == 'UP':
                p = [start[j][0], start[j][1], start[j][2]+1]
                n_list.append(p)
            if path[i] == 'DOWN':
                p = [start[j][0], start[j][1], start[j][2]-1]
                n_list.append(p)
            if path[i] == 'CW' or path[i] == 'CCW':
                p = rot_point(start[j], path[i])
                n_list.append(p)
        start = n_list[len(n_list)-3], n_list[len(n_list)-2],n_list[len(n_list)-1]
    
    return n_list



if __name__ == "__main__":
    start, goal, rp_start1, rp_start2, rp_start3, rp_goal1, rp_goal2, rp_goal3 = get_setup()
    
    print("start: \n", start)
    print("goal: \n", goal)
    path, steps = astar(start, goal)
    print("path:", path)

    n_list = (points(start, goal, path))
    for i in range(int(len(n_list)/3)):
        print(f"Centroid Path {i+1}:", n_list[3*i],n_list[3*i+1], n_list[3*i+2] )
    
    print("\n")
    pathreg1 = region(path, rp_start1)
    for i in range(len(pathreg1)):
        print(f"Reg Points {i}:", pathreg1[i])
        
    print("\n")
    pathreg2 = region(path, rp_start2)
    for i in range(len(pathreg2)):
        print(f"Reg Points {i}:", pathreg2[i])
        
    print("\n")
    pathreg3 = region(path, rp_start3)
    for i in range(len(pathreg3)):
        print(f"Reg Points {i}:", pathreg3[i])

start: 
 [[-1.25  0.   -2.5 ]
 [ 0.   -1.25 -2.5 ]
 [ 1.25  0.   -2.5 ]]
goal: 
 [[ 1.25  0.    1.5 ]
 [ 0.    1.25  1.5 ]
 [-1.25  0.    1.5 ]]
path: ['UP', 'UP', 'UP', 'UP', 'CW', 'CW']
Centroid Path 1: [-1.25, 0.0, -1.5] [0.0, -1.25, -1.5] [1.25, 0.0, -1.5]
Centroid Path 2: [-1.25, 0.0, -0.5] [0.0, -1.25, -0.5] [1.25, 0.0, -0.5]
Centroid Path 3: [-1.25, 0.0, 0.5] [0.0, -1.25, 0.5] [1.25, 0.0, 0.5]
Centroid Path 4: [-1.25, 0.0, 1.5] [0.0, -1.25, 1.5] [1.25, 0.0, 1.5]
Centroid Path 5: [ 0.   -1.25  1.5 ] [1.25 0.   1.5 ] [0.   1.25 1.5 ]
Centroid Path 6: [1.25 0.   1.5 ] [0.   1.25 1.5 ] [-1.25  0.    1.5 ]


Reg Points 0: [[-1.25, -1.25, -3], [-1.25, -1.25, -2], [-1.25, 1.25, -2], [-1.25, 1.25, -3]]
Reg Points 1: [(-1.25, -1.25, -2), (-1.25, -1.25, -1), (-1.25, 1.25, -1), (-1.25, 1.25, -2)]
Reg Points 2: [(-1.25, -1.25, -1), (-1.25, -1.25, 0), (-1.25, 1.25, 0), (-1.25, 1.25, -1)]
Reg Points 3: [(-1.25, -1.25, 0), (-1.25, -1.25, 1), (-1.25, 1.25, 1), (-1.25, 1.25, 0)]
Reg Points 4: [(